# Programa de Pós-graduação em Ciência da Computação (PPGCC) da UFSCar
# Disciplina: Processamento Digital de Imagens 3D e Vídeos
## Professor: Ricardo José Ferrari
### Alunos: 
### Adriano dos Reis Carvalho
### Rodrigo César Evangelista 
# TRABALHO PRÁTICO

## 1 Tarefa
### Quantificação volumétrica das substâncias branca (WM), cinzenta (GM) e líquido cefalorraquidiano (CSF) do cérebro humano em imagens de Ressonância Magnética (MR), utilizando segmentação não supervisionada (K- Médias) e atlas probabilísticos dos tecidos cerebrais.

## 2 Materiais e Métodos
## 2.1 Imagens
### Esse projeto fará uso de cinco imagens de imagens de RM T1-weighted (se diz, T1-ponderada), pois fornecem uma melhor definição da anatomia do cérebro humano, e de um atlas probabilístico dos 3 principais tecidos cerebrais (GM, WM e CSF). As imagens que compõem o atlas probabilístico estão alinhadas espacialmente com a imagem de referência T1-w e possuem níveis de cinza que variam entre 0 e 1. Os níveis de cinza em cada posição espacial (x,y,z) se completam para a probabilidade de 100%. Ou seja, para uma dada posição espacial (x,y,z) tem-se: Px,y,z (WM) + Px,y,z (GM) + Px,y,z (CSF) = 1
## 2.2 Métodos
### O desenvolvimento do projeto deverá ser realizado usando a linguagem python (versão 3.12 ou maior) e a entrega deverá ser feita na forma de um arquivo jupyter notebook. O aluno deverá criar e utilizar um ambiente virtual e, juntamente com a entrega do notebook, o aluno deverá também entregar o arquivo “requirements.txt” (“pip3 freeze > requirements.txt”) para que eu possa reproduzir o ambiente virtual e avaliar o projeto. O projeto que não for funcional, ou seja, se os métodos não puderem ser executados por erro, receberá nota ZERO. A seguir, é indicada a sequência de etapas que deverá ser implementada.

## 2.3 Sequência de etapas
### Com base nas técnicas apresentadas em aula, o aluno deverá implementar funções em python3 para a realização das seguintes etapas do pipeline do projeto:

## • 1a Etapa
### – Redução de ruído das imagens (denoising).
### – O aluno deve avaliar os resultados da redução de ruído nas imagens utilizando os algoritmos Filtro Bilateral, Filtro de Difusão Anisotrópica e Filtro de Média Não-Local. A avaliação deve ser qualitativa, escolhendo o método vitorioso com base na redução de ruído enquanto preserva os detalhes da imagem. Isso pode ser feito analisando o valor absoluto da diferença entre as imagens, ou seja, G = |Ioriginal − Idenoised|. Se a imagem G contiver estruturas nítidas e coerentes com a anatomia do cérebro, isso indica que, além do ruído, o algoritmo também removeu informações importantes da imagem. Por outro lado, se a imagem Idenoised contiver ruídos visíveis, isso indica que o algoritmo não foi eficaz na remoção do ruído.

## Desenvolvimento da 1a Etapa

### Função

In [2]:
import os
def read_directories(directory, img=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)          
        else:
            filenames.append(filename)    
    return filenames

### Diretório das imagens

In [3]:
dir_images = f'work1/clinical_images'
array_images = read_directories(dir_images)

### – O aluno deve avaliar os resultados da redução de ruído nas imagens utilizando os algoritmos Filtro Bilateral, Filtro de Difusão Anisotrópica e Filtro de Média Não-Local.

### Aplicando o Filtro Anisotrópico

In [4]:
import SimpleITK as sitk
import time

def apply_anisotropic_diffusion_sitk(image_path, time_step, conductance, iterations, output_path):
    """
    Apply a 3D anisotropic diffusion filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - time_step (float): The time step for the diffusion process.
    - conductance (float): The conductance parameter for the diffusion process.
    - iterations (int): Number of iterations for the diffusion process.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Cast the image to a supported pixel type (float32)
    image = sitk.Cast(image, sitk.sitkFloat32)
    
    # Execution time
    inicio = time.time()

    # Apply Anisotropic Diffusion filter
    # Create an instance of the anisotropic diffusion filter
    anisotropic_diffusion_filter = sitk.CurvatureAnisotropicDiffusionImageFilter()
    # Set the time step for the filter, ensuring stability of the algorithm
    anisotropic_diffusion_filter.SetTimeStep(time_step)
    # Set the conductance parameter, controlling the sensitivity to edges in the image
    anisotropic_diffusion_filter.SetConductanceParameter(conductance)
    # Set the number of iterations for the diffusion process
    anisotropic_diffusion_filter.SetNumberOfIterations(iterations)
    # Apply the filter to the input image and obtain the filtered result
    filtered_image = anisotropic_diffusion_filter.Execute(image)
    
    # Execution time
    fim = time.time()
    tempo = fim - inicio
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")
    print('Execution time: %d seconds \n'%(tempo))


# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/anisotropicDiffusion'  
    time_step, conductance, iterations =  0.0585, 1.0, 10
    #time_step, conductance, iterations =  0.0625, 1.0, 10 #Fiz alteração no item time_step

    for image in array_images:    
        output_name = f'{image}'
        apply_anisotropic_diffusion_sitk(f'{dir_images}/{image}', time_step, conductance, iterations, f'{dir_output}/{output_name}')
        

Filtered image using SimpleITK saved to work1/denoising/anisotropicDiffusion/IXI012-HH-1211-T1.nii.gz
Execution time: 5 seconds 

Filtered image using SimpleITK saved to work1/denoising/anisotropicDiffusion/IXI013-HH-1212-T1.nii.gz
Execution time: 6 seconds 

Filtered image using SimpleITK saved to work1/denoising/anisotropicDiffusion/IXI015-HH-1258-T1.nii.gz
Execution time: 5 seconds 

Filtered image using SimpleITK saved to work1/denoising/anisotropicDiffusion/IXI002-Guys-0828-T1.nii.gz
Execution time: 5 seconds 

Filtered image using SimpleITK saved to work1/denoising/anisotropicDiffusion/IXI014-HH-1236-T1.nii.gz
Execution time: 5 seconds 



### Aplicando o Filtro Non Local Means (Skimage)

In [5]:
import numpy as np
from skimage.restoration import denoise_nl_means, estimate_sigma
import nibabel as nib
from pathlib import Path

def apply_nonlocal_means_skimage(image_path, patch_size, patch_distance, h, fast_mode, output_path):
    """
    Apply a 3D Non-Local Means filter to a 3D image using scikit-image.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - patch_size (int): The size of the patches used for denoising.
    - patch_distance (int): The distance for the neighborhood search.
    - h (float): The filtering parameter.
    - fast_mode (bool): Whether to use the fast mode or not.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = nib.load(image_path)
    image_data = image.get_fdata()
    
    # Estimate the noise standard deviation from the noisy image
    sigma_est = np.mean(estimate_sigma(image_data))

    # Execution time
    inicio = time.time()

    # Apply Non-Local Means filter
    # image_data: The input image to be denoised.
    # patch_size: Size of the patches used for comparison in the filter (e.g., 7x7 pixels).
    # patch_distance: Maximum distance between patches used in the filtering.
    # h: Filtering strength, controlling the degree of smoothing. Often scaled by the estimated noise level (`sigma_est`).
    # fast_mode: If `True`, uses a faster approximation algorithm at the cost of slightly reduced accuracy.
    denoised_image = denoise_nl_means(image_data, patch_size=patch_size, patch_distance=patch_distance, h=h*sigma_est, fast_mode=fast_mode)

    # Execution time
    fim = time.time()
    tempo = fim - inicio

    # Save the filtered image
    denoised_image_nib = nib.Nifti1Image(denoised_image, image.affine)
    nib.save(denoised_image_nib, output_path)
    print(f"Filtered image using scikit-image saved to {output_path}")
    print('Execution time: %d seconds \n'%(tempo))

# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/non_local_means'
    i = 0
    patch_size, patch_distance, h = 3, 5, 1.15
    for image in array_images:    
        output_name = f'{image}'
        apply_nonlocal_means_skimage(f'{dir_images}/{image}',  patch_size, patch_distance, h, True, f'{dir_output}/{output_name}')

Filtered image using scikit-image saved to work1/denoising/non_local_means/IXI012-HH-1211-T1.nii.gz
Execution time: 136 seconds 

Filtered image using scikit-image saved to work1/denoising/non_local_means/IXI013-HH-1212-T1.nii.gz
Execution time: 137 seconds 

Filtered image using scikit-image saved to work1/denoising/non_local_means/IXI015-HH-1258-T1.nii.gz
Execution time: 140 seconds 

Filtered image using scikit-image saved to work1/denoising/non_local_means/IXI002-Guys-0828-T1.nii.gz
Execution time: 146 seconds 

Filtered image using scikit-image saved to work1/denoising/non_local_means/IXI014-HH-1236-T1.nii.gz
Execution time: 141 seconds 



### Aplicando o Filtro Bilateral

In [6]:
import SimpleITK as sitk

def apply_bilateral_filter_sitk(image_path, domain_sigma, range_sigma, output_path):
    """
    Apply a 3D bilateral filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - domain_sigma (float): The standard deviation for the spatial Gaussian kernel.
    - range_sigma (float): The standard deviation for the range Gaussian kernel.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Execution time
    inicio = time.time()

    # Apply Bilateral filter
    # Create an instance of the Bilateral filter
    bilateral_filter = sitk.BilateralImageFilter()
    # Set the domain sigma, which determines the spatial extent of the filter
    bilateral_filter.SetDomainSigma(domain_sigma)
    # Set the range sigma, which controls how differences in intensity values are treated
    bilateral_filter.SetRangeSigma(range_sigma)
    # Apply the filter to the input image and obtain the filtered result
    filtered_image = bilateral_filter.Execute(image)

    # Execution time
    fim = time.time()
    tempo = fim - inicio

    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")
    print('Execution time: %d seconds \n'%(tempo))

if __name__ == "__main__":
    array_images = read_directories(dir_images)
    dir_output = f'work1/denoising/bilateral'
    domain_sigma, range_sigma =  2.0, 50.0   

for image in array_images:    
    output_name = f't{image}'
    apply_bilateral_filter_sitk(f'{dir_images}/{image}', domain_sigma, range_sigma, f'{dir_output}/{output_name}')    


Filtered image using SimpleITK saved to work1/denoising/bilateral/tIXI012-HH-1211-T1.nii.gz
Execution time: 28 seconds 

Filtered image using SimpleITK saved to work1/denoising/bilateral/tIXI013-HH-1212-T1.nii.gz
Execution time: 28 seconds 

Filtered image using SimpleITK saved to work1/denoising/bilateral/tIXI015-HH-1258-T1.nii.gz
Execution time: 27 seconds 

Filtered image using SimpleITK saved to work1/denoising/bilateral/tIXI002-Guys-0828-T1.nii.gz
Execution time: 26 seconds 

Filtered image using SimpleITK saved to work1/denoising/bilateral/tIXI014-HH-1236-T1.nii.gz
Execution time: 28 seconds 



## G - Images

### A avaliação deve ser qualitativa, escolhendo o método vitorioso com base na redução de ruído enquanto preserva os detalhes da imagem. Isso pode ser feito analisando o valor absoluto da diferença entre as imagens, ou seja, G = |Ioriginal − Idenoised|. Se a imagem G contiver estruturas nítidas e coerentes com a anatomia do cérebro, isso indica que, além do ruído, o algoritmo também removeu informações importantes da imagem. Por outro lado, se a imagem Idenoised contiver ruídos visíveis, isso indica que o algoritmo não foi eficaz na remoção do ruído.

#### bilateral

In [ ]:
dir_bilateral = f'work1/denoising/bilateral/images_filtered'
dir_out_bilateral = f'work1/denoising/bilateral/G'

# Execution time
inicio = time.time()

for image in array_images:
    # Read the original image from the specified directory
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    # Read the filtered image (from bilateral filtering) from the specified directory
    filtered_image = sitk.ReadImage(f'{dir_bilateral}/{image}')
    # Convert the original image to a NumPy array for processing
    image_array = sitk.GetArrayFromImage(original_image)
    # Convert the filtered image to a NumPy array for processing
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    # Compute the absolute difference between the original and filtered images
    image_g = abs(image_array-image_array_filtered)
    # Convert the difference array back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(image_g)
    # Save the resulting difference image to the specified output directory
    sitk.WriteImage(filtered_image, f'{dir_out_bilateral}/{image}')

# Execution time
fim = time.time()
tempo = fim - inicio
print('Execution time: %d seconds \n'%(tempo))

#### anisotropicDiffusion

In [ ]:
dir_anisotropic = f'work1/denoising/anisotropicDiffusion/images_filtered'
dir_out_anisotropic = f'work1/denoising/anisotropicDiffusion/G'

# Execution time
inicio = time.time()

for image in array_images:
    # Read the original image from the specified directory
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    # Read the filtered image (from anisotropic filtering) from the specified directory
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    # Convert the original image to a NumPy array for processing
    image_array = sitk.GetArrayFromImage(original_image)
    # Convert the filtered image to a NumPy array for processing
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    # Compute the absolute difference between the original and filtered images
    image_g = abs(image_array-image_array_filtered)
    # Convert the difference array back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(image_g)
    # Save the resulting difference image to the specified output directory
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')

# Execution time
fim = time.time()
tempo = fim - inicio
print('Execution time: %d seconds \n'%(tempo))


#### non_local_means

In [ ]:
dir_nlm = f'work1/denoising/non_local_means/images_filtered'
dir_out_nlm = f'work1/denoising/non_local_means/G'

# Execution time
inicio = time.time()

for image in array_images:
    # Read the original image from the specified directory
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    # Read the filtered image (from Non Local Means filtering) from the specified directory
    filtered_image = sitk.ReadImage(f'{dir_nlm}/{image}')
    # Convert the original image to a NumPy array for processing
    image_array = sitk.GetArrayFromImage(original_image)
    # Convert the filtered image to a NumPy array for processing
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    # Compute the absolute difference between the original and filtered images
    image_g = abs(image_array-image_array_filtered)
    # Convert the difference array back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(image_g)
    # Save the resulting difference image to the specified output directory
    sitk.WriteImage(filtered_image, f'{dir_out_nlm}/{image}')

# Execution time
fim = time.time()
tempo = fim - inicio
print('Execution time: %d seconds \n'%(tempo))
